In [64]:
import praw
import pandas as pd
import time
from datetime import datetime, timezone


In [ ]:
#initialize Reddit instance
reddit = praw.Reddit(client_id="kN6wpKi6buf6GQUyH6EuZw",
                     client_secret="WbK11cNfVdk6gvswQC-8cZDPxqYLKw",
                     user_agent="python:myredditapp:v1.0 (by u/NewIntroduction1834)")

In [ ]:
subreddit_name = "news"
search_term = "trump coin"
start_date = datetime(2025, 1, 17, tzinfo=timezone.utc)
end_date = datetime(2025, 2, 28, tzinfo=timezone.utc)
post_limit = None  

#convert dates to Unix timestamps
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

#search posts
subreddit = reddit.subreddit(subreddit_name)
posts = subreddit.search(search_term, limit=post_limit)

#collect all comments in a list
all_comments = []

for idx, post in enumerate(posts, start=1):
    post_datetime = datetime.fromtimestamp(post.created_utc, tz=timezone.utc)
    if post_datetime < start_date or post_datetime > end_date:
        continue

    thread_title = post.title
    thread_url = f"https://www.reddit.com{post.permalink}"
    thread_time_utc = post_datetime.strftime('%Y-%m-%d %H:%M:%S')
    thread_score = post.score

    print(f"{idx}. Processing: {thread_title} | {thread_time_utc} | Score: {thread_score}")

    post.comments.replace_more(limit=100)
    for comment in post.comments.list():
        comment_time_utc = datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        comment_text = comment.body.replace('\n', ' ').strip()
        comment_score = comment.score

        all_comments.append({
            "thread_title": thread_title,
            "thread_url": thread_url,
            "thread_time_utc": thread_time_utc,
            "thread_score": thread_score,
            "comment_time_utc": comment_time_utc,
            "comment_score": comment_score,
            "comment_text": comment_text
        })

    #add delay every 100 threads
    if idx % 100 == 0:
        print("Sleeping for 3 seconds to respect Reddit API limits...")
        time.sleep(3)

#save results to CSV
df = pd.DataFrame(all_comments)
df = df.sort_values(by="comment_time_utc")
df.to_csv(f"{subreddit_name}_{search_term}.csv", index=False, encoding="utf-8")

print(f"\n Done! Saved {len(df)} comments from threads with '{search_term}' to '{subreddit_name}_{search_term}.csv'")


3. Processing: Bybit Hit by Crypto’s Worst Hack With Almost $1.5 Billion Stolen | 2025-02-21 19:35:44 | Score: 198

✅ Done! Saved 21 comments from threads with 'trump coin' to 'subreddit_keyword_comments_filtered.csv'


C:\Users\User\AppData\Local\Temp\ipykernel_1296\964921405.py:33: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_time_utc = datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')


In [ ]:
#merging CSV files

import pandas as pd
import os

folder_path = 'Reddit_Data/melania'

#list all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

#read and merge all CSV files into one DataFrame
df_list = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

#concatenate all DataFrames in the list
merged_df = pd.concat(df_list, ignore_index=True)

#save the merged DataFrame to a new CSV file
merged_df.to_csv(f'{folder_path}_merged.csv', index=False, encoding='utf-8')

print("CSV files merged and saved as 'merged_output.csv'")


CSV files merged and saved as 'merged_output.csv'
